In [8]:

#############################################################################################  
#### Palani's Data code Start
#############################################################################################  
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

#Age Group classification
Age_group_bins=[0, 4, 7, 9, 13, 17, 18]
Age_group_Names=["0-19 Years", "20-34 Years", "35-44 Years", "45-64 Years", "65-84 Years", "85 and older"]

#############################################################################################  
#Get US National data for 2000_2009 API data
url_us_2000 = "https://api.census.gov/data/2000/pep/int_charagegroups?get=DATE_DESC,DATE,GEONAME,POP,AGEGROUP,SEX&for=us:1"
us_response_2000 = requests.get(url_us_2000)
us_2000_2009_data = us_response_2000.json()

# Add it dataframe and convert to text fields to numeric
df_us_2000_2009_data = pd.DataFrame(us_2000_2009_data[1:], columns=us_2000_2009_data[0]).rename(columns = {"DATE_DESC":"PERIOD_DESC", "DATE":"PERIOD", "us":"US"})
df_us_2000_2009_data["POP"] = pd.to_numeric(df_us_2000_2009_data["POP"])
df_us_2000_2009_data["AGEGROUP"] = pd.to_numeric(df_us_2000_2009_data["AGEGROUP"])
df_us_2000_2009_data["PERIOD"] = pd.to_numeric(df_us_2000_2009_data["PERIOD"])
df_us_2000_2009_data["SEX"] = pd.to_numeric(df_us_2000_2009_data["SEX"])
#df_us_2000_2009_data.head(1000)


# filtering agegroup(0 - 85) and period -2000(2000-2009(2-11) and sex(male-1, and female-2))
df_us_2000_2009_AgeGroup= df_us_2000_2009_data.loc[df_us_2000_2009_data["AGEGROUP"].isin(range(1,19)) 
                                                   & (df_us_2000_2009_data["PERIOD"].between(2,11) ) 
                                                   & (df_us_2000_2009_data["SEX"].between(1,2)), :]

#year
df_us_2000_2009_AgeGroup["Year"] =  pd.to_numeric(df_us_2000_2009_AgeGroup["PERIOD_DESC"].str.slice(4,9))
df_us_2000_2009_AgeGroup.head()

#binning age group
df_us_2000_2009_AgeGroup["AgeGroup_Names"] = pd.cut(df_us_2000_2009_AgeGroup["AGEGROUP"], Age_group_bins, labels=Age_group_Names)
    

  
# combining us national data (2000 to 2017)
#Get US National data for 2000_2009 API data
url_us_2010_2017 = "https://api.census.gov/data/2017/pep/charagegroups?get=DATE_DESC,DATE,GEONAME,POP,AGEGROUP,SEX&for=us:1"

us_response_2010_2017 = requests.get(url_us_2010_2017)
us_2010_2017_data = us_response_2010_2017.json()

# Add it dataframe and convert to text fields to numeric
df_us_2010_2017_data = pd.DataFrame(us_2010_2017_data[1:], columns=us_2010_2017_data[0]).rename(columns = {"DATE_DESC":"PERIOD_DESC", "DATE":"PERIOD", "us":"US"})
df_us_2010_2017_data["POP"] = pd.to_numeric(df_us_2010_2017_data["POP"])
df_us_2010_2017_data["AGEGROUP"] = pd.to_numeric(df_us_2010_2017_data["AGEGROUP"])
df_us_2010_2017_data["PERIOD"] = pd.to_numeric(df_us_2010_2017_data["PERIOD"])
df_us_2010_2017_data["SEX"] = pd.to_numeric(df_us_2010_2017_data["SEX"])
df_us_2010_2017_data.head(1000)

# filtering agegroup(0 - 85) and period (2010-2017(2-11) and sex(male-1, and female-2))
df_us_2010_2017_AgeGroup= df_us_2010_2017_data.loc[df_us_2010_2017_data["AGEGROUP"].isin(range(1,19)) 
                                                   & (df_us_2010_2017_data["PERIOD"].between(3,11) ) 
                                                   & (df_us_2010_2017_data["SEX"].between(1,2)), :]

#year
df_us_2010_2017_AgeGroup["Year"] =  pd.to_numeric(df_us_2010_2017_AgeGroup["PERIOD_DESC"].str.slice(4,9))
df_us_2010_2017_AgeGroup.head()

#binning age group
df_us_2010_2017_AgeGroup["AgeGroup_Names"] = pd.cut(df_us_2010_2017_AgeGroup["AGEGROUP"], Age_group_bins, labels=Age_group_Names)
df_us_2010_2017_AgeGroup.head()

df_us_combine_2000_2017 = pd.concat([df_us_2000_2009_AgeGroup, df_us_2010_2017_AgeGroup], ignore_index=True)
df_us_combine_2000_2017.tail()

grp_us_combine_2000_2017 = df_us_combine_2000_2017.groupby(["AgeGroup_Names", "Year"])

df_plt_us_combine_2000_2017 = pd.DataFrame({"Sum":grp_us_combine_2000_2017["POP"].sum()}).reset_index()

AgeGroup_types = df_plt_us_combine_2000_2017.groupby("AgeGroup_Names")["AgeGroup_Names"].max()
years = df_plt_us_combine_2000_2017.groupby("Year")["Year"].max()

net_pop_change = []

for agegrp in AgeGroup_types:
    net_change = 0
    prev_sum = 0 
    for year in years:
        sum = list(df_plt_us_combine_2000_2017.loc[(df_plt_us_combine_2000_2017["AgeGroup_Names"] == agegrp) & 
                                                    (df_plt_us_combine_2000_2017["Year"]==year), "Sum"])[0]
        if year == 2000:
            net_change = 0
            prev_sum = sum
            #print(f" {agegrp} : {year}: Sum:{sum} : Prev_sum{prev_sum} ")
        else:
            #print(f" {agegrp} : {year}: Sum:{sum} : Prev_sum{prev_sum} ")
            net_change = ((sum - prev_sum)/prev_sum) * 100
            prev_sum = sum
            
        net_pop_change.append(round(net_change,4))
        
#print(net_pop_change)  
       
df_plt_us_combine_2000_2017["net_change"]=net_pop_change

df_plt_us_combine = df_plt_us_combine_2000_2017.loc[df_plt_us_combine_2000_2017["Year"]>2000, :]
#df_plt_us_combine.head(1000)

df_plt_us_combine.to_csv("./Data/df_plt_us_combine.csv")


#############################################################################################  

#Get Seattle Metro data for 2000_2009 API data
url_seattle_metro_2000 = "https://api.census.gov/data/2000/pep/int_charagegroups?get=DATE_DESC,DATE,GEONAME,POP,AGEGROUP,SEX&for=county:053,033,061&in=state:53"

seattle_metro_response_2000 = requests.get(url_seattle_metro_2000)
seattle_metro_2000_2009_data = seattle_metro_response_2000.json()

# Add it dataframe and convert to text fields to numeric
df_seattle_metro_2000_2009_data = pd.DataFrame(seattle_metro_2000_2009_data[1:], columns=seattle_metro_2000_2009_data[0]).rename(columns = {"DATE_DESC":"PERIOD_DESC", "DATE":"PERIOD"})
df_seattle_metro_2000_2009_data["POP"] = pd.to_numeric(df_seattle_metro_2000_2009_data["POP"])
df_seattle_metro_2000_2009_data["AGEGROUP"] = pd.to_numeric(df_seattle_metro_2000_2009_data["AGEGROUP"])
df_seattle_metro_2000_2009_data["PERIOD"] = pd.to_numeric(df_seattle_metro_2000_2009_data["PERIOD"])
df_seattle_metro_2000_2009_data["SEX"] = pd.to_numeric(df_seattle_metro_2000_2009_data["SEX"])

#df_seattle_metro_2000_2009_data.head(1000)
# filtering agegroup(0 - 85) and period -2000(2000-2009(2-11) and sex(male-1, and female-2))
df_seattle_metro_2000_2009_AgeGroup= df_seattle_metro_2000_2009_data.loc[df_seattle_metro_2000_2009_data["AGEGROUP"].isin(range(1,19)) 
                                                   & (df_seattle_metro_2000_2009_data["PERIOD"].between(2,11) ) 
                                                   & (df_seattle_metro_2000_2009_data["SEX"].between(1,2)), :]

df_seattle_metro_2000_2009_AgeGroup["AgeGroup_Names"] = pd.cut(df_seattle_metro_2000_2009_AgeGroup["AGEGROUP"], Age_group_bins, labels=Age_group_Names)
df_seattle_metro_2000_2009_AgeGroup["Year"] =  pd.to_numeric(df_seattle_metro_2000_2009_AgeGroup["PERIOD_DESC"].str.slice(4,9))

url_seattle_metro_2017 = "https://api.census.gov/data/2017/pep/charagegroups?get=DATE_DESC,DATE,GEONAME,POP,AGEGROUP,SEX&for=county:053,033,061&in=state:53"
seattle_metro_response_2017 = requests.get(url_seattle_metro_2017)
seattle_metro_2010_2017_data = seattle_metro_response_2017.json()

# Add it dataframe and convert to text fields to numeric
df_seattle_metro_2010_2017_data = pd.DataFrame(seattle_metro_2010_2017_data[1:], columns=seattle_metro_2010_2017_data[0]).rename(columns = {"DATE_DESC":"PERIOD_DESC", "DATE":"PERIOD"})
df_seattle_metro_2010_2017_data["POP"] = pd.to_numeric(df_seattle_metro_2010_2017_data["POP"])
df_seattle_metro_2010_2017_data["AGEGROUP"] = pd.to_numeric(df_seattle_metro_2010_2017_data["AGEGROUP"])
df_seattle_metro_2010_2017_data["PERIOD"] = pd.to_numeric(df_seattle_metro_2010_2017_data["PERIOD"])
df_seattle_metro_2010_2017_data["SEX"] = pd.to_numeric(df_seattle_metro_2010_2017_data["SEX"])

#df_seattle_metro_2010_2017_data.head(1000)
# filtering agegroup(0 - 85) and period -2000(2000-2009(2-11) and sex(male-1, and female-2))
df_seattle_metro_2010_2017_AgeGroup= df_seattle_metro_2010_2017_data.loc[df_seattle_metro_2010_2017_data["AGEGROUP"].isin(range(1,19)) 
                                                   & (df_seattle_metro_2010_2017_data["PERIOD"].between(3,11) ) 
                                                   & (df_seattle_metro_2010_2017_data["SEX"].between(1,2)), :]

df_seattle_metro_2010_2017_AgeGroup["AgeGroup_Names"] = pd.cut(df_seattle_metro_2010_2017_AgeGroup["AGEGROUP"], Age_group_bins, labels=Age_group_Names)
df_seattle_metro_2010_2017_AgeGroup["Year"] =  pd.to_numeric(df_seattle_metro_2010_2017_AgeGroup["PERIOD_DESC"].str.slice(4,9))

df_seattle_metro_2010_2017_AgeGroup.head(100)
#df_seattle_metro_2010_2017_AgeGroup.to_csv("df_seattle_metro_2010_2017_AgeGroup.csv")
df_seattle_combine_2000_2017 = pd.concat([df_seattle_metro_2000_2009_AgeGroup, df_seattle_metro_2010_2017_AgeGroup],ignore_index=True)

df_seattle_combine_2000_2017.tail()

grp_seattle_combine_2000_2017 = df_seattle_combine_2000_2017.groupby(["AgeGroup_Names", "Year"])

df_plt_seattle_combine_2000_2017 = pd.DataFrame({"Sum":grp_seattle_combine_2000_2017["POP"].sum()}).reset_index()

AgeGroup_types = df_plt_seattle_combine_2000_2017.groupby("AgeGroup_Names")["AgeGroup_Names"].max()
years = df_plt_seattle_combine_2000_2017.groupby("Year")["Year"].max()

net_pop_change = []

for agegrp in AgeGroup_types:
    net_change = 0
    prev_sum = 0 
    for year in years:
        sum = list(df_plt_seattle_combine_2000_2017.loc[(df_plt_seattle_combine_2000_2017["AgeGroup_Names"] == agegrp) & 
                                                    (df_plt_seattle_combine_2000_2017["Year"]==year), "Sum"])[0]
        if year == 2000:
            net_change = 0
            prev_sum = sum
            #print(f" {agegrp} : {year}: Sum:{sum} : Prev_sum{prev_sum} ")
        else:
            #print(f" {agegrp} : {year}: Sum:{sum} : Prev_sum{prev_sum} ")
            net_change = ((sum - prev_sum)/prev_sum) * 100
            prev_sum = sum
            
        net_pop_change.append(round(net_change,4))
        
#print(net_pop_change)  
       
df_plt_seattle_combine_2000_2017["net_change"]=net_pop_change

df_plt_seattle_combine = df_plt_seattle_combine_2000_2017.loc[df_plt_seattle_combine_2000_2017["Year"]>2000, :]
#df_plt_seattle_combine.head(1000)
df_plt_seattle_combine.to_csv("./Data/df_plt_seattle_combine.csv")

###############################################################################################
# Data and Graph for gender
# Us National


grp_us_combine_2000_2017_gender = df_us_combine_2000_2017.groupby(["SEX", "Year"])

df_plt_us_combine_2000_2017_gender = pd.DataFrame({"Sum":grp_us_combine_2000_2017_gender["POP"].sum()}).reset_index()

gender_types = df_plt_us_combine_2000_2017_gender.groupby("SEX")["SEX"].max()
years = df_plt_us_combine_2000_2017_gender.groupby("Year")["Year"].max()

net_pop_change = []

for gender in gender_types:
    net_change = 0
    prev_sum = 0 
    for year in years:
        sum = list(df_plt_us_combine_2000_2017_gender.loc[(df_plt_us_combine_2000_2017_gender["SEX"] == gender) & 
                                                    (df_plt_us_combine_2000_2017["Year"]==year), "Sum"])[0]
        if year == 2000:
            net_change = 0
            prev_sum = sum
            
        else:
            
            net_change = ((sum - prev_sum)/prev_sum) * 100
            prev_sum = sum
            
        net_pop_change.append(round(net_change,4))       

       
df_plt_us_combine_2000_2017_gender["net_change"]=net_pop_change

df_plt_us_combine_gender = df_plt_us_combine_2000_2017_gender.loc[df_plt_us_combine_2000_2017_gender["Year"]>2000, :]


df_plt_us_combine_gender.to_csv("./Data/df_plt_us_combine_gender.csv")

# Data and Graph for gender
# Seattle Metro


grp_seattle_combine_2000_2017_gender = df_seattle_combine_2000_2017.groupby(["SEX", "Year"])

df_plt_seattle_combine_2000_2017_gender = pd.DataFrame({"Sum":grp_seattle_combine_2000_2017_gender["POP"].sum()}).reset_index()

gender_types = df_plt_seattle_combine_2000_2017_gender.groupby("SEX")["SEX"].max()
years = df_plt_seattle_combine_2000_2017_gender.groupby("Year")["Year"].max()

net_pop_change = []

for gender in gender_types:
    net_change = 0
    prev_sum = 0 
    for year in years:
        sum = list(df_plt_seattle_combine_2000_2017_gender.loc[(df_plt_seattle_combine_2000_2017_gender["SEX"] == gender) & 
                                                    (df_plt_seattle_combine_2000_2017["Year"]==year), "Sum"])[0]
        if year == 2000:
            net_change = 0
            prev_sum = sum
            
        else:
            
            net_change = ((sum - prev_sum)/prev_sum) * 100
            prev_sum = sum
            
        net_pop_change.append(round(net_change,4))       

       
df_plt_seattle_combine_2000_2017_gender["net_change"]=net_pop_change

df_plt_seattle_combine_gender = df_plt_seattle_combine_2000_2017_gender.loc[df_plt_seattle_combine_2000_2017_gender["Year"]>2000, :]

df_plt_seattle_combine_gender.to_csv("./Data/df_plt_seattle_combine_gender.csv")

#############################################################################################  
#### Palani's Data code End
#############################################################################################  

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexi